# Movie Quotes Corpus EDA

# 1. Cleaning

In [ ]:
# #---mount googledrive to tensorflow--
# from google.colab import drive
# drive.mount('/content/gdrive')

# #---Upload files to tensorflow--
# from google.colab import files
# uploaded = files.upload()

In [10]:
import pandas as pd
import fileinput

In [6]:
with open('./data/cornell_movie_dialogs_corpus/README.txt', encoding = "ISO-8859-1") as f:
    print(f.read())

f.close()

Cornell Movie-Dialogs Corpus

Distributed together with:

"Chameleons in imagined conversations: A new approach to understanding coordination of linguistic style in dialogs"
Cristian Danescu-Niculescu-Mizil and Lillian Lee
Proceedings of the Workshop on Cognitive Modeling and Computational Linguistics, ACL 2011.

(this paper is included in this zip file)

NOTE: If you have results to report on these corpora, please send email to cristian@cs.cornell.edu or llee@cs.cornell.edu so we can add you to our list of people using this data.  Thanks!


Contents of this README:

	A) Brief description
	B) Files description
	C) Details on the collection procedure
	D) Contact


A) Brief description:

This corpus contains a metadata-rich collection of fictional conversations extracted from raw movie scripts:

- 220,579 conversational exchanges between 10,292 pairs of movie characters
- involves 9,035 characters from 617 movies
- in total 304,713 utterances
- movie metadata included:
	- genres
	- relea

In [27]:
def read_txt(filename):
    with fileinput.FileInput(filename, inplace=True, backup='.bak') as file:
        for line in file:
            print(line)
            try:
                print(line.replace(" +++$+++ ", "\t"), end='')
            except:
                line.encode('utf-8').strip()
                print(line.replace(" +++$+++ ", "\t"), end='')

In [35]:
# read_txt('/tf/lec/Galvanize/ResumeClassifier/data/cornell_movie_dialogs_corpus/origin_txt/movie_characters_metadata.txt')
# read_txt('movie_conversations.txt')
# read_txt('movie_titles_metadata.txt')
# read_txt('movie_lines.txt')

In [14]:
import sys
sys.path.insert(0,'/tf/lec/Galvanize/ResumeClassifier/src')

In [24]:
from movie_quotes_corpus_EDA import txt_to_df

In [56]:
# characters_metadata_cols = ['characterID', 'character_name', 'movieID', 'movieTitle', 'gender', 'position_in_credits']
# conversations_cols = ["characterID_First", "characterID_second", "movieID", "list_of_utterances"]
# titles_metadata_cols = ['movieID', 'movieTitle', 'movieYear', 'IMDB_rating', 'numIMDBvotes', 'Genres']
# lines_cols = ['lineID', 'characterID', 'movieID', 'character_name', 'text_of_utterances']


# characters_metadata_df = txt_to_df('movie_characters_metadata.txt', characters_metadata_cols)
# conversations_df = txt_to_df('movie_conversations.txt', conversations_cols)
# titles_metadata_df = txt_to_df('movie_titles_metadata.txt', titles_metadata_cols)
# lines_df = txt_to_df('movie_lines.txt', lines_cols)

None

In [ ]:
# characters_metadata_df.to_csv('movie_characters_metadata.csv', index=False)
# conversations_df.to_csv('movie_conversations.csv', index=False)
# titles_metadata_df.to_csv('movie_titles_metadata.csv', index=False)
# lines_df.to_csv('movie_lines.csv', index=False)

In [92]:
characters_metadata_df = pd.read_csv('/tf/lec/Galvanize/ResumeClassifier/data/cornell_movie_dialogs_corpus/create_csv/movie_characters_metadata.csv')
characters_metadata_df.head()

,characterID,character_name,movieID,movieTitle,gender,position_in_credits
0,u0,BIANCA,m0,10 things i hate about you,f,4
1,u1,BRUCE,m0,10 things i hate about you,?,?
2,u2,CAMERON,m0,10 things i hate about you,m,3
3,u3,CHASTITY,m0,10 things i hate about you,?,?
4,u4,JOEY,m0,10 things i hate about you,m,6


In [86]:
conversations_df = pd.read_csv('/tf/lec/Galvanize/ResumeClassifier/data/cornell_movie_dialogs_corpus/create_csv/movie_conversations.csv')
conversations_df.head()

,characterID_First,characterID_second,movieID,list_of_utterances
0,u0,u2,m0,"['L194', 'L195', 'L196', 'L197']"
1,u0,u2,m0,"['L198', 'L199']"
2,u0,u2,m0,"['L200', 'L201', 'L202', 'L203']"
3,u0,u2,m0,"['L204', 'L205', 'L206']"
4,u0,u2,m0,"['L207', 'L208']"


In [104]:
titles_metadata_df = pd.read_csv('/tf/lec/Galvanize/ResumeClassifier/data/cornell_movie_dialogs_corpus/create_csv/movie_titles_metadata.csv')
titles_metadata_df.head()

,movieID,movieTitle,movieYear,IMDB_rating,numIMDBvotes,Genres
0,m0,10 things i hate about you,1999,6.9,62847,"['comedy', 'romance']"
1,m1,1492: conquest of paradise,1992,6.2,10421,"['adventure', 'biography', 'drama', 'history']"
2,m2,15 minutes,2001,6.1,25854,"['action', 'crime', 'drama', 'thriller']"
3,m3,2001: a space odyssey,1968,8.4,163227,"['adventure', 'mystery', 'sci-fi']"
4,m4,48 hrs.,1982,6.9,22289,"['action', 'comedy', 'crime', 'drama', 'thrill..."


In [111]:
lines_df = pd.read_csv('/tf/lec/Galvanize/ResumeClassifier/data/cornell_movie_dialogs_corpus/create_csv/movie_lines.csv')
lines_df.head()

,lineID,characterID,movieID,character_name,text_of_utterances
0,L1045,u0,m0,BIANCA,They do not!
1,L1044,u2,m0,CAMERON,They do to!
2,L985,u0,m0,BIANCA,I hope so.
3,L984,u2,m0,CAMERON,She okay?
4,L925,u0,m0,BIANCA,Let's go.


In [112]:
characters_metadata_df.shape, conversations_df.shape, titles_metadata_df.shape, lines_df.shape

((1706, 6), (83097, 4), (74, 6), (5401, 5))

In [113]:
# lines_df.drop_duplicates(subset ="lineID", keep = 'first', inplace = True)
# lines_df.to_csv('/tf/lec/Galvanize/ResumeClassifier/data/cornell_movie_dialogs_corpus/create_csv/movie_lines.csv', index=False)

In [110]:
groupby_character_lines_df = lines_df.groupby(['movieID', 'characterID'])

In [122]:
temp = lines_df.groupby(['movieID', 'characterID','character_name'])['text_of_utterances'].apply(list)

In [126]:
temp.values

array([list(['They do not!', 'I hope so.', "Let's go.", "Okay -- you're gonna need to learn how to lie.", 'I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?', 'Like my fear of wearing pastels?', 'What good stuff?', "Me.  This endless ...blonde babble. I'm like, boring myself.", 'do you listen to this crap?', 'Then Guillermo says, "If you go any lighter, you\'re gonna look like an extra on 90210."', 'But', 'Well, no...', 'I was?', 'Tons', 'You know Chastity?', 'Hi.', "Who knows?  All I've ever heard her say is that she'd dip before dating a guy that smokes.", "Lesbian?  No. I found a picture of Jared Leto in one of her drawers, so I'm pretty sure she's not harboring same-sex tendencies.", "I really, really, really wanna go, but I can't.  Not unless my sister goes.", "Eber's Deep Conditioner every two days. And I never, ever use a blowdryer without the diffuser attachment.", "You're sweet.", "I counted on you to help my cause. You and

In [142]:
temp_lines_df_text = lines_df.groupby(['movieID', 'characterID','character_name'])['text_of_utterances'].apply(list)
temp_lines_df_lineid = lines_df.groupby(['movieID', 'characterID','character_name'])['lineID'].apply(list)

In [143]:
temp_lines_df_text.shape, temp_lines_df_lineid.shape

((203,), (203,))

In [144]:
temp_lines_df_text_0 = temp_lines_df_text.reset_index()
temp_lines_df_lineid_0 = temp_lines_df_lineid.reset_index()

In [146]:
temp_lines_df_text_0.shape, temp_lines_df_lineid_0.shape

((203, 4), (203, 4))

In [147]:
temp_lines_df_text_0.head()

,movieID,characterID,character_name,text_of_utterances
0,m0,u0,BIANCA,"[They do not!, I hope so., Let's go., Okay -- ..."
1,m0,u1,BRUCE,"[Just sent 'em through., Never, Didn't have yo..."
2,m0,u10,SHARON,"[Absolutely not., Your daughters went to the p..."
3,m0,u11,WALTER,"[I'm missing something., You're not going unle..."
4,m0,u2,CAMERON,"[They do to!, She okay?, Wow, No, The ""real yo..."


In [148]:
temp_lines_df_lineid_0.head()

,movieID,characterID,character_name,lineID
0,m0,u0,BIANCA,"[L1045, L985, L925, L872, L870, L869, L867, L8..."
1,m0,u1,BRUCE,"[L463, L461, L459]"
2,m0,u10,SHARON,"[L932, L930, L926, L918, L886, L876, L192, L17..."
3,m0,u11,WALTER,"[L922, L883, L881, L879, L546, L544, L540, L53..."
4,m0,u2,CAMERON,"[L1044, L984, L924, L871, L868, L866, L865, L8..."


In [163]:
temp_lines_df_lineid_0['lineID']

0      [L1045, L985, L925, L872, L870, L869, L867, L8...
1                                     [L463, L461, L459]
2      [L932, L930, L926, L918, L886, L876, L192, L17...
3      [L922, L883, L881, L879, L546, L544, L540, L53...
4      [L1044, L984, L924, L871, L868, L866, L865, L8...
5        [L952, L660, L600, L598, L596, L579, L572, L50]
6      [L759, L758, L756, L593, L591, L589, L396, L39...
7      [L1052, L1022, L1010, L1008, L983, L913, L911,...
8      [L948, L946, L755, L753, L751, L720, L718, L71...
9      [L779, L585, L583, L499, L432, L424, L419, L41...
10     [L1018, L109, L107, L105, L103, L511, L263, L2...
11     [L462, L460, L458, L780, L745, L743, L741, L72...
12     [L2181, L2179, L2176, L2175, L2173, L2172, L2170]
13     [L2025, L2022, L2019, L2015, L2011, L2005, L19...
14     [L2539, L2538, L2473, L2472, L2319, L2123, L21...
15     [L2432, L2430, L2425, L2423, L2421, L2292, L2290]
16     [L2024, L2023, L2020, L2016, L2014, L2012, L20...
17     [L2552, L2550, L2548, L2

In [181]:
lines_groupby_df = pd.merge(temp_lines_df_lineid_0, temp_lines_df_text_0['text_of_utterances'], left_index=True, right_index=True, how='outer')
lines_groupby_df.head()

,movieID,characterID,character_name,lineID,text_of_utterances
0,m0,u0,BIANCA,"[L1045, L985, L925, L872, L870, L869, L867, L8...","[They do not!, I hope so., Let's go., Okay -- ..."
1,m0,u1,BRUCE,"[L463, L461, L459]","[Just sent 'em through., Never, Didn't have yo..."
2,m0,u10,SHARON,"[L932, L930, L926, L918, L886, L876, L192, L17...","[Absolutely not., Your daughters went to the p..."
3,m0,u11,WALTER,"[L922, L883, L881, L879, L546, L544, L540, L53...","[I'm missing something., You're not going unle..."
4,m0,u2,CAMERON,"[L1044, L984, L924, L871, L868, L866, L865, L8...","[They do to!, She okay?, Wow, No, The ""real yo..."


In [182]:
# lines_groupby_df.to_csv('lines_groupby.csv', index= False)

In [183]:
pwd

'/tf/lec/Galvanize/ResumeClassifier/data/cornell_movie_dialogs_corpus/create_csv'

In [185]:
lines_groupby_df = pd.read_csv('lines_groupby.csv')
lines_groupby_df.head()

,movieID,characterID,character_name,lineID,text_of_utterances
0,m0,u0,BIANCA,"['L1045', 'L985', 'L925', 'L872', 'L870', 'L86...","['They do not!', 'I hope so.', ""Let's go."", ""O..."
1,m0,u1,BRUCE,"['L463', 'L461', 'L459']","[""Just sent 'em through."", 'Never', ""Didn't ha..."
2,m0,u10,SHARON,"['L932', 'L930', 'L926', 'L918', 'L886', 'L876...","['Absolutely not.', 'Your daughters went to th..."
3,m0,u11,WALTER,"['L922', 'L883', 'L881', 'L879', 'L546', 'L544...","[""I'm missing something."", ""You're not going u..."
4,m0,u2,CAMERON,"['L1044', 'L984', 'L924', 'L871', 'L868', 'L86...","['They do to!', 'She okay?', 'Wow', 'No', 'The..."


In [186]:
lines_groupby_df.shape

(203, 5)